In [58]:
using Dates
using JSON3
using DataFrames

In [59]:
firstDate = Dates.DateTime(2023, 07, 19,   21, 0, 0)

2023-07-19T21:00:00

In [60]:
lastDate  = Dates.DateTime(2024, 01, 20,    9, 0, 0)

2024-01-20T09:00:00

In [61]:
measurementDate = collect(firstDate:Dates.Hour(6):lastDate);

In [62]:
filenames = ["/home/lali/TITAN-ROG-sync/python/METEO/MET-Norway-"*i*".json" for i in string.(measurementDate)]
println("You have ", size(filenames)[1], " MET-Norway files   ->   ", last(filenames))

You have 739 MET-Norway files   ->   /home/lali/TITAN-ROG-sync/python/METEO/MET-Norway-2024-01-20T09:00:00.json


In [63]:
function readOneFile(js)
    f = open(js,"r") 
    inDict = JSON3.read(f)
    close(f)
    
    return inDict
end 

readOneFile (generic function with 1 method)

In [64]:
myDict = readOneFile(filenames[1])
@show nowTime = myDict.RO.now
@show nowTemp = myDict.RO.temp

nowTime = myDict.RO.now = "2023-07-19T21:00:00"
nowTemp = myDict.RO.temp = 24.8


24.8

In [65]:
size(myDict.METNO)[1]

38

In [66]:
[convert(Dates.Hour, Dates.DateTime(myDict.METNO[D].now)  - Dates.DateTime(nowTime)) for D in 1:39]

BoundsError: BoundsError: attempt to access 38-element JSON3.Array{JSON3.Object, Base.CodeUnits{UInt8, String}, SubArray{UInt64, 1, Vector{UInt64}, Tuple{UnitRange{Int64}}, true}} at index [39]

In [67]:
anticipateValues = [string(Dates.value(convert(Dates.Hour, Dates.DateTime(myDict.METNO[D].now)  - Dates.DateTime(nowTime) ))) for D in 1:size(myDict.METNO)[1]]

38-element Vector{String}:
 "0"
 "6"
 "12"
 "18"
 "24"
 "30"
 "36"
 "42"
 "48"
 "54"
 ⋮
 "174"
 "180"
 "186"
 "192"
 "198"
 "204"
 "210"
 "216"
 "222"

In [68]:
allData = DataFrame([name => Float64[] for name in anticipateValues])

Row,0,6,12,18,24,30,36,42,48,54,60,66,72,78,84,90,96,102,108,114,120,126,132,138,144,150,156,162,168,174,180,186,192,198,204,210,216,222
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64


In [69]:
insertcols!(allData, 1, :Data => DateTime[])
insertcols!(allData, 2, :Real => Float64[])
insertcols!(allData,    :"228" => Float64[]) # primul fișier are mai puține rânduri
allowmissing!(allData)

Row,Data,Real,0,6,12,18,24,30,36,42,48,54,60,66,72,78,84,90,96,102,108,114,120,126,132,138,144,150,156,162,168,174,180,186,192,198,204,210,216,222,228
,DateTime?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?


In [70]:
#câte un rând pentru fiecare fișier
for measurement in measurementDate
    push!(allData, [measurement; missing; [missing for _ in anticipateValues]; missing])
end
#plus cele viitoare
for future in 1:size(myDict.METNO)[1]
    #println(lastDate + Hour(future*6))
    push!(allData, [lastDate + Hour(future*6); missing; [missing for _ in anticipateValues]; missing])
end

Empty DF ready!

In [71]:
#filename = filenames[10]
for filename in filenames
    myDict = readOneFile(filename)
    nowTime = Dates.DateTime(myDict.RO.now)
    nowTemp = myDict.RO.temp
    #println("La data de: ", nowTime, "  a fost: ", nowTemp)
    myRow = Int(Dates.value(convert(Dates.Hour, nowTime  - firstDate))/6+1)
    allData[myRow,:Real] = nowTemp
    for i in myDict.METNO
        myRow = Int(Dates.value(convert(Dates.Hour, Dates.DateTime(i.now)  - firstDate))/6+1)
        myCol = string(Dates.value(convert(Dates.Hour, Dates.DateTime(i.now)  - nowTime)))
        #println("[", myRow, " ", myCol, "]: ", i.now, "  ", i.temp)
        allData[myRow,myCol] = i.temp
    end
end

DRAW

In [72]:
x = [parse(Int, x) for x in anticipateValues]
y = allData[:,:Data]
z = fill(NaN, size(y, 1), size(x, 1))
for i in eachindex(x), j in eachindex(y)
    myRow = Int(Dates.value(convert(Dates.Hour, Dates.DateTime(y[j])  - firstDate))/6+1)
    if ismissing(allData[myRow, string(x[i])])
        z[j, i] = 0.0
    else
        z[j, i] = allData[myRow, string(x[i])]
    end
end

In [73]:
#using Plots

In [74]:
#heatmap(x, y, z)

In [75]:
using CairoMakie

In [76]:
f = Figure()
Axis(f[1, 1])

co = CairoMakie.contourf!(x, y, z)

Colorbar(f[1, 2], co)

f

StackOverflowError: StackOverflowError: